Importing Libraries:
==

In [28]:
%matplotlib inline

import time
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch import optim

from sklearn.metrics import jaccard_score as jsc

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')
    device = "cuda"
    torch.set_default_tensor_type('torch.cuda.FloatTensor') 
else:
    print('GPU is not available!')
    device = "cpu"

GPU is not available!


Loading Data:
==

In [29]:
input_channels = 3
max_channels = 512
output_classes = 1


batch_size = 16
train_size = 100 # divide train:val by 70:30 probably
test_size = 0
# there are 100 images of 750x750x3 size
# https://drive.google.com/drive/folders/1KoVt9vK6Prf4JYW3c3jfm4aSwqQXdWc0 has readme \
    # and all data
# use nn.utils.data.Dataset and similar stuff perhaps and maybe check
# https://github.com/mateuszbuda/brain-segmentation-pytorch/blob/master/dataset.py


Creating Module:
==

In [30]:
class UNetConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv2 = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(output_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv2(x)
class DownConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv_down = nn.Sequential(
            nn.MaxPool2d(2, 2),
            UNetConv(input_channels, output_channels)
        )
    def forward(self, x):
        return self.conv_down(x)
class UpConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv_up = nn.Sequential(
            nn.ConvTranspose2d(input_channels , input_channels // 2, kernel_size=2, stride=2),
        )
        self.conv_level = nn.Sequential(
            UNetConv(input_channels,output_channels)
        )
    def forward(self, x1, x2):
        return self.conv_level(torch.cat([x2,self.conv_up(x1)],dim = 1))
class LastConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv_final = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=1)
        )
    def forward(self, x):
        return self.conv_final(x)
class UNet(nn.Module):
    def __init__(self, input_channels, max_channels):
        super(UNet, self).__init__()
        self.current_channels = max_channels // 2**4
        self.start = UNetConv(input_channels, self.current_channels)
        self.current_channels = self.current_channels*2
        self.down1 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels*2
        self.down2 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels*2
        self.down3 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels*2
        self.down4 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up1 = UpConv(self.current_channels * 2, self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up2 = UpConv(self.current_channels * 2, self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up3 = UpConv(self.current_channels * 2, self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up4 = UpConv(self.current_channels * 2, self.current_channels)
        self.final = LastConv(self.current_channels,output_classes)
        
    def forward(self, x):
        x = self.start(x)
        x = self.down4(self.down3(self.down2(self.down1(x))))
        x = self.up4(self.up3(self.up2(self.up1(x))))
        return self.final(x)

Model = UNet(input_channels, max_channels)

Training Model:
==

In [31]:
epochs = 5
loss_func = jsc
optimizer = optim.Adam(Model.parameters(),lr=0.01)

trainLoss = []
for epoch in range(epochs):  
    epochStart = time.time()
    runningLoss = 0.0
    for inputs, labels in tqdm(train_dataLoader): 
# need to add the train data via a data loader like \
# https://github.com/mateuszbuda/brain-segmentation-pytorch/blob/master/dataset.py
# or using nn.utils.data.Dataset 

        optimizer.zero_grad()  
        outputs = Model(inputs)  
        loss = loss_func(outputs, labels)
        loss.backward() 
        nn.utils.clip_grad_norm_(Model.parameters(), 50)
        optimizer.step()
        runningLoss += loss.item()
        
    runningLoss /= len(input) 
    trainLoss.append(runningLoss)
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ; Time consumed: {:.0f}m {:.0f}s '\
        .format(epoch + 1, epochs, runningLoss, epochEnd//60, epochEnd%60))   
print('Finished Training')

NameError: name 'train_dataLoader' is not defined